In [2]:
import numpy as np, pandas as pd, pathlib

# ----- parámetros -----
N_MAT, N_RUNS = 10_000, 10
PERC = [0.1,1,2,3,4,5,10,20,25,50]
RNG  = np.random.default_rng(0)           # semilla reproducible
OUT  = pathlib.Path("resultados_fuzzy"); OUT.mkdir(exist_ok=True)

def random_fuzzy(n):
    """(N_MAT, n, n) con diag 0.5 y U(0.0001,0.9999) off-diag."""
    m = RNG.uniform(1e-4, 0.9999, size=(N_MAT, n, n))
    idx = np.arange(n)
    m[:, idx, idx] = 0.5
    return m

def nonreciprocity(a, p_norm):
    # |log( (f_ij f_ji)/((1-f_ij)(1-f_ji)) )|
    triu = np.triu_indices(a.shape[1], k=1)
    f_ij     = a[:, triu[0], triu[1]]
    f_ji     = a[:, triu[1], triu[0]]
    t        = (f_ij * f_ji) / ((1-f_ij)*(1-f_ji))
    t        = np.clip(t, 1e-10, 1e10)
    diffs    = np.abs(np.log(t))          # usa np.log10 si lo necesitas
    if p_norm == 1:
        return diffs.sum(axis=1)
    if p_norm == 2:
        return np.linalg.norm(diffs, axis=1)
    if p_norm == np.inf:
        return diffs.max(axis=1)
    raise ValueError("p_norm debe ser 1, 2 o np.inf")

def tabla_percentiles(n, p_norm, fname):
    filas = []
    for perc in PERC:
        vals = [np.percentile(nonreciprocity(random_fuzzy(n), p_norm), perc)
                for _ in range(N_RUNS)]
        filas.append({"percentile": perc, **{f"Run {i+1}": v for i,v in enumerate(vals)},
                      "Avg": np.mean(vals)})
    pd.DataFrame(filas).to_excel(fname, index=False)
    print("  ✓", fname.name)

for n in range(2, 9):
    print(f"n = {n}")
    tabla_percentiles(n, 1,      OUT / f"fuzzy_{n}x{n}_p1.xlsx")
    tabla_percentiles(n, 2,      OUT / f"fuzzy_{n}x{n}_p2.xlsx")
    tabla_percentiles(n, np.inf, OUT / f"fuzzy_{n}x{n}_pInf.xlsx")

print("\n✔ Simulación completada.")



n = 2
  ✓ fuzzy_2x2_p1.xlsx
  ✓ fuzzy_2x2_p2.xlsx
  ✓ fuzzy_2x2_pInf.xlsx
n = 3
  ✓ fuzzy_3x3_p1.xlsx
  ✓ fuzzy_3x3_p2.xlsx
  ✓ fuzzy_3x3_pInf.xlsx
n = 4
  ✓ fuzzy_4x4_p1.xlsx
  ✓ fuzzy_4x4_p2.xlsx
  ✓ fuzzy_4x4_pInf.xlsx
n = 5
  ✓ fuzzy_5x5_p1.xlsx
  ✓ fuzzy_5x5_p2.xlsx
  ✓ fuzzy_5x5_pInf.xlsx
n = 6
  ✓ fuzzy_6x6_p1.xlsx
  ✓ fuzzy_6x6_p2.xlsx
  ✓ fuzzy_6x6_pInf.xlsx
n = 7
  ✓ fuzzy_7x7_p1.xlsx
  ✓ fuzzy_7x7_p2.xlsx
  ✓ fuzzy_7x7_pInf.xlsx
n = 8
  ✓ fuzzy_8x8_p1.xlsx
  ✓ fuzzy_8x8_p2.xlsx
  ✓ fuzzy_8x8_pInf.xlsx

✔ Simulación completada.


In [1]:
#!/usr/bin/env python3
# ------------------------------------------------------------
# Monte Carlo para no-reciprocidad en matrices aditivas
# Autor:       Luis Ángel Calvo Pascual (ejemplo)
# Descripción: genera tablas de percentiles (p = 1, 2, ∞)
# ------------------------------------------------------------

import numpy as np
import pandas as pd
from pathlib import Path

# ----------------------------- Parámetros globales ----------
N_MATRICES  = 10_000          # matrices por ejecución
N_RUNS      = 10              # ejecuciones independientes
PERCENTILES = [0.1, 1, 2, 3, 4, 5, 10, 20, 25, 50]

LOW, HIGH   = -100, 100       # rango uniforme U(-100, 100)
OUTPUT_DIR  = Path("resultados_additive")
OUTPUT_DIR.mkdir(exist_ok=True)

# ----------------------------- Generación de matrices -------
def generate_random_additive_matrices(n_matrices: int, n: int,
                                      low: float = LOW, high: float = HIGH) -> np.ndarray:
    """
    Devuelve un array de forma (n_matrices, n, n) con valores ~ U(low, high).
    """
    return np.random.uniform(low, high, size=(n_matrices, n, n))

# ----------------------------- No-reciprocidad aditiva ------
def calculate_non_reciprocity_additive(a: np.ndarray,
                                       norm: float | int = 1,
                                       normalize: bool = False) -> np.ndarray:
    """
    Calcula la no-reciprocidad de cada matriz en 'a' según |a_ij + a_ji|.
    
    Parámetros
    ----------
    a : ndarray de forma (n_matrices, n, n)
    norm : 1, 2 o np.inf
    normalize : si True divide por el nº de pares (n*(n-1)/2)
    
    Devuelve
    --------
    ndarray de longitud n_matrices con la medida para cada matriz.
    """
    n = a.shape[1]
    # Obtener índices únicos i < j
    idx_pairs = [(i, j) for i in range(n) for j in range(i + 1, n)]
    
    # |a_ij + a_ji| para cada par
    diffs = [np.abs(a[:, i, j] + a[:, j, i]) for i, j in idx_pairs]
    diffs = np.stack(diffs, axis=1)              # (n_matrices, n_pairs)
    
    if norm == 1:
        nonrec = np.sum(diffs, axis=1)
    elif norm == 2:
        nonrec = np.linalg.norm(diffs, axis=1)
    elif norm == np.inf:
        nonrec = np.max(diffs, axis=1)
    else:
        raise ValueError("Norma no soportada. Usa 1, 2 o np.inf.")
    
    if normalize:
        nonrec /= len(idx_pairs)
    return nonrec

# ----------------------------- Generar y guardar tablas -----
def generate_results_for_norm(n: int, norm: float | int, filename: Path,
                              normalize: bool = False) -> None:
    """
    Ejecuta N_RUNS simulaciones para matrices n×n, calcula los percentiles
    y guarda un Excel con resultados individuales y promedio.
    """
    records = []
    for perc in PERCENTILES:
        valores_run = []
        for _ in range(N_RUNS):
            matrices = generate_random_additive_matrices(N_MATRICES, n)
            nonrec   = calculate_non_reciprocity_additive(matrices, norm, normalize)
            valores_run.append(np.percentile(nonrec, perc))
        
        # Fila con runs + promedio
        fila = {"percentile": perc, **{f"Run {i+1}": v for i, v in enumerate(valores_run)},
                "Avg": np.mean(valores_run)}
        records.append(fila)
    
    df = pd.DataFrame(records)
    df.to_excel(filename, index=False)
    print(f"   → Guardado: {filename.name}")

# ----------------------------- Bloque principal -------------
if __name__ == "__main__":
    for n in range(2, 9):                         # tamaños 2 … 8
        print(f"\nProcesando n = {n}")
        generate_results_for_norm(
            n, 1,      OUTPUT_DIR / f"additive_{n}x{n}_p1.xlsx")
        generate_results_for_norm(
            n, 2,      OUTPUT_DIR / f"additive_{n}x{n}_p2.xlsx")
        generate_results_for_norm(
            n, np.inf, OUTPUT_DIR / f"additive_{n}x{n}_pInf.xlsx")

    print("\n✔ Simulación completada.")




Procesando n = 2
   → Guardado: additive_2x2_p1.xlsx
   → Guardado: additive_2x2_p2.xlsx
   → Guardado: additive_2x2_pInf.xlsx

Procesando n = 3
   → Guardado: additive_3x3_p1.xlsx
   → Guardado: additive_3x3_p2.xlsx
   → Guardado: additive_3x3_pInf.xlsx

Procesando n = 4
   → Guardado: additive_4x4_p1.xlsx
   → Guardado: additive_4x4_p2.xlsx
   → Guardado: additive_4x4_pInf.xlsx

Procesando n = 5
   → Guardado: additive_5x5_p1.xlsx
   → Guardado: additive_5x5_p2.xlsx
   → Guardado: additive_5x5_pInf.xlsx

Procesando n = 6
   → Guardado: additive_6x6_p1.xlsx
   → Guardado: additive_6x6_p2.xlsx
   → Guardado: additive_6x6_pInf.xlsx

Procesando n = 7
   → Guardado: additive_7x7_p1.xlsx
   → Guardado: additive_7x7_p2.xlsx
   → Guardado: additive_7x7_pInf.xlsx

Procesando n = 8
   → Guardado: additive_8x8_p1.xlsx
   → Guardado: additive_8x8_p2.xlsx
   → Guardado: additive_8x8_pInf.xlsx

✔ Simulación completada.
